### **import packages**

In [75]:
import sklearn
import pandas as pd
import re
import torch
import torch.nn as nn
import json
from torch.utils.data import Dataset, DataLoader
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#  = 'cpu'

### **email preprocessing and labelling**

In [31]:
with open("emails.txt") as f: 
    unlabelled = f.read().split('\n')

In [32]:
# import re
# def is_valid_email(email):
#     """Check if the email is a valid format."""

#     # Regular expression for validating an Email

#     # regex = r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w+$'
#     # regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
#     # regex = re.compile(r"^[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?$" )
#     regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'

#     # If the string matches the regex, it is a valid email

#     if re.match(regex, email):

#         return True

#     else:

#         return False


In [54]:
# # some emails cause errors because they're invalid
# new_emails = 'email\n'
# for email in unlabelled:
#     if is_valid_email(email):
#         new_emails = new_emails + f'{email}\n'

In [55]:
# with open('purified.txt', 'w+') as f:
#     f.write(new_emails)


In [90]:
# df = pd.read_csv("purified.txt")

In [91]:
# create age and label columns
template = re.compile(r'\d+')

for idx, row in df.iterrows():
    potential_age = template.search(row.email)
    if potential_age is None:
        age = -100
        df.loc[idx, "label"] = 0
    else:
        date_as_str = potential_age.group(0)
        date_as_int = int(date_as_str)
        if len(date_as_str) == 2:
            if date_as_int <= 24:
                age = 24 - date_as_int
                df.loc[idx, "label"] = 1
            else:
                age = 124 - date_as_int
                if age >= 50:
                    df.loc[idx, "label"] = 3
                else:
                    df.loc[idx, "label"] = 2
        elif len(date_as_str) == 4:
            if date_as_int >= 2006 or date_as_int <= 1900: # assuming everyone is 18+
                age = -100
                df.loc[idx, "label"] = 0
            else:
                age = 2024 - date_as_int
                if age >= 50:
                        df.loc[idx, "label"] = 3
                elif age < 30:
                    df.loc[idx, "label"] = 1
                else:
                    df.loc[idx, "label"] = 2
        else:
            age = -100
            df.loc[idx, "label"] = 0
    df.loc[idx,"age"] = age
                

In [92]:
# df.to_csv("purified_and_labelled.csv")

In [93]:
# df["stripped"] = df["email"].apply(lambda x: x.split("@")[0].lower())

In [99]:
# char_to_int = {
#     "0": 1,
#     "1": 2,
#     "2": 3,
#     "3": 4,   
#     "4":5,
#     "5":6,
#     "6": 7,
#     "7": 8,
#     "8": 9,
#     "9": 10,
#     ".": 11,
#     "_": 12,
#     "-": 13
# }

In [100]:
# abc = "abcdefghijklmnopqrstuvwxyz"

In [101]:
# for i, c in enumerate(abc):
#     char_to_int[c] = i + 14

In [111]:
# with open('mapping.json', 'w+') as f:
#     json.dump(char_to_int, f)

In [103]:
# df['features'] = df["stripped"].apply(lambda x: [char_to_int[d] for d in x] )

In [105]:
# longest = 0
# for _, row in df.iterrows():
#     if len(row.features) > longest:
#         longest = len(row.features)
# longest

30

In [107]:
# input to the networks must be uniform
# def pad_tokens(token_list):
#     len_tokens = len(token_list)
#     if len_tokens > 32 :
#         return token_list[:32]
#     else:
#         pad_val = [0] * 32
#         extended = token_list + pad_val # length of (original_length + max_len)
#         return extended[:-len_tokens]

In [108]:
# df['padded'] = df['features'].apply(pad_tokens)

In [109]:
# df.to_csv("emails_labelled.csv")


In [110]:
# train_data = df.iloc[0:800]
# valid_data = df.iloc[800:900]
# test_data = df.iloc[900:]
# test_data.to_csv('test_data.csv')
# valid_data.to_csv('validation_data.csv')
# train_data.to_csv("training_data.csv")

### **Model building**

In [53]:
df = pd.read_csv('emails_labelled.csv')

In [54]:
features = df.padded.apply(lambda x: eval(x)).values
labels = df.label.values

In [70]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=1)

(803,)


In [99]:

forest = RandomForestClassifier()
forest.fit(list(x_train), y_train)

RandomForestClassifier()

In [102]:
y_forest = forest.predict(list(x_test))
predictions = forest.predict_proba(list(x_test))
print("forest Accuracy:",metrics.accuracy_score(y_test, y_forest))

forest Accuracy: 0.7313432835820896


In [2]:
class Emails(Dataset):
        def __init__(self, csv_dir):
            data = pd.read_csv(csv_dir)
            self.features = data.padded.apply(lambda x: eval(x)).values
            self.labels = data.label.values
        def __len__(self):
            return len(self.labels)
        def __getitem__(self, idx):
            return self.features[idx], self.labels[idx]

In [3]:
train_ds = Emails("training_data.csv")
valid_ds = Emails("validation_data.csv")

In [103]:
from joblib import dump, load
dump(forest, 'forest_model.joblib') 



['forest_model.joblib']

In [104]:
with open("mapping.json", 'r') as f:
    mapping = json.load(f)


In [119]:
score_labels = {}

score_labels[0] = 'undefined'
score_labels[1] = 'young'
score_labels[2] = 'medium'
score_labels[3] = 'old'


with open("scores.json", 'w+') as f:
    json.dump(score_labels,f)


### **evaluation pipeline build**

In [109]:
# input to the networks must be uniform
def pad_tokens(token_list: list):
    len_tokens = len(token_list)
    if len_tokens > 32 :
        return token_list[:32]
    else:
        pad_val = [0] * 32
        extended = token_list + pad_val # length of (original_length + max_len)
        return extended[:-len_tokens]

In [110]:
def tokenize_and_pad(email: str):
    email = email.split('@')[0].lower()
    tokens = [mapping[ch] for ch in email]
    tokens = pad_tokens(tokens)
    return tokens

In [146]:
predictions = forest.predict_proba([processed])
label = np.argmax(predictions)
score = np.max(predictions)
print(label, score)

3 0.39


### **Pytorch model testing**

In [49]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # self.fc1 = nn.Linear(32, 64)
        # self.fc2 = nn.Linear(64, 16)
        # self.fc1 = nn.Linear(32, 8)
        # self.fc2 = nn.Linear(8, 4)
        self.conv1 = nn.Conv1d(1, 16, kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv1d(16, 1, kernel_size= 5, stride = 4, padding = 1 )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        x = torch.unsqueeze(x, dim=1)
        # print(x.shape)
        x = self.relu(self.conv1(x))
        # x = self.relu(self.fc2(x))
        # x = self.relu(self.fc3(x))
        # x = self.fc4(x)
        x = self.conv2(x)
        return x.squeeze(dim=1)

In [50]:
lr = 1e-3
epochs = 50
batch_size = 16
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= lr)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,drop_last=True)
valid_dataloader = DataLoader(valid_ds, batch_size=1, shuffle=True)

In [51]:
for epoch in range(epochs):
    training_loss = 0
    validation_loss = 0
    validation_accuracy = 0
    for batch in train_dataloader:
        model.train()
        optimizer.zero_grad()
        features, labels = batch
        features = torch.stack(features).float()
        features = torch.transpose(features, 0, 1).to(device)
        labels = labels.to(device).long()
        outputs = model(features)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        
    model.eval()
    with torch.no_grad():
        for batch in valid_dataloader:
            features, labels = batch
            features = torch.stack(features).float()
            features = torch.transpose(features, 0, 1).to(device)
            labels = labels.to(device).long()
            outputs = model(features)
            loss = loss_fn(outputs, labels)
            validation_loss += loss.item()
            if torch.argmax(outputs).item() == labels.item():
                validation_accuracy += 1
    print(f' losses at {epoch+1} epoch(s): {training_loss / len(train_dataloader)} -- {validation_loss / len(valid_dataloader)} -- {validation_accuracy / len(valid_dataloader)}')
    # print(f'training loss at {epoch+1} epoch(s): {epoch_loss / len(train_dataloader)}')

 losses at 1 epoch(s): 1.761562008857727 -- 1.4403756871819495 -- 0.35
 losses at 2 epoch(s): 1.3852433252334595 -- 1.3456395751237868 -- 0.4
 losses at 3 epoch(s): 1.3292412090301513 -- 1.3649610176682472 -- 0.36
 losses at 4 epoch(s): 1.3114346289634704 -- 1.3733655819296837 -- 0.36
 losses at 5 epoch(s): 1.3005125713348389 -- 1.3297643366456031 -- 0.39
 losses at 6 epoch(s): 1.2894523000717164 -- 1.4213209956884385 -- 0.38
 losses at 7 epoch(s): 1.298942506313324 -- 1.3274223935604095 -- 0.42
 losses at 8 epoch(s): 1.2804566025733948 -- 1.317525379061699 -- 0.41
 losses at 9 epoch(s): 1.2940052032470704 -- 1.3401930725574493 -- 0.39
 losses at 10 epoch(s): 1.2777458453178405 -- 1.3397457346320152 -- 0.4
 losses at 11 epoch(s): 1.2805321383476258 -- 1.3195379880070686 -- 0.41
 losses at 12 epoch(s): 1.279609878063202 -- 1.3819944925606251 -- 0.38
 losses at 13 epoch(s): 1.2798580873012542 -- 1.332159004509449 -- 0.39
 losses at 14 epoch(s): 1.2728362560272217 -- 1.309211387038231 -- 